# Example full chip design

In [1]:
%load_ext autoreload
%autoreload 2

# Import Modules

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

In [3]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.line_tee import LineTee

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

from collections import OrderedDict

from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

## Define "TransmonPocket_sqnl"

In [4]:
from qiskit_metal.qlibrary.core import BaseQubit
import numpy as np
class TransmonPocket_sqnl(BaseQubit):
    default_options = Dict(
        pad_gap='30um',
        inductor_width='20um',
        pad_width='455um',
        pad_height='90um',
        pocket_width='650um',
        pocket_height='650um',
        # 90 has dipole aligned along the +X axis,
        # while 0 has dipole aligned along the +Y axis
        _default_connection_pads=Dict(
            connector_type='0',  # 0 = Claw type, 1 = T-shape
            pad_width='125um',
            pad_height='30um',
            pad_cpw_shift='0um',
            pad_cpw_extent='25um',
            claw_length='30um',
            claw_width='10um',
            claw_gap='6um',
            claw_cpw_length='40um',
            claw_cpw_width='10um',
            ground_spacing='5um',
            connector_location= '0'  # 0~5 means six locations with rotated CCW starting from right-top site
        ))
    """Default drawing options"""

    component_metadata = Dict(short_name='Pocket_claw',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""

    TOOLTIP = """The base `TransmonPocket_sqnl` class."""

    def make(self):
        """Define the way the options are turned into QGeometry.

        The make function implements the logic that creates the geoemtry
        (poly, path, etc.) from the qcomponent.options dictionary of
        parameters, and the adds them to the design, using
        qcomponent.add_qgeometry(...), adding in extra needed
        information, such as layer, subtract, etc.
        """
        self.make_pocket()
        self.make_connection_pads()

    def make_pocket(self):
        """Makes standard transmon in a pocket."""

        # self.p allows us to directly access parsed values (string -> numbers) form the user option
        p = self.p

        # extract chip name
        chip = p.chip

        # since we will reuse these options, parse them once and define them as varaibles
        pad_width = p.pad_width
        pad_height = p.pad_height
        pad_gap = p.pad_gap

        # make the pads as rectangles (shapely polygons)
        pad = draw.rectangle(pad_width, pad_height)
        pad_top = draw.translate(pad, 0, +(pad_height + pad_gap) / 2.)
        pad_bot = draw.translate(pad, 0, -(pad_height + pad_gap) / 2.)

        rect_jj = draw.LineString([(0, -pad_gap / 2), (0, +pad_gap / 2)])
        # the draw.rectangle representing the josephson junction
        # rect_jj = draw.rectangle(p.inductor_width, pad_gap)

        rect_pk = draw.rectangle(p.pocket_width, p.pocket_height)

        # Rotate and translate all qgeometry as needed.
        # Done with utility functions in Metal 'draw_utility' for easy rotation/translation
        # NOTE: Should modify so rotate/translate accepts qgeometry, would allow for
        # smoother implementation.
        polys = [rect_jj, pad_top, pad_bot, rect_pk]
        polys = draw.rotate(polys, p.orientation, origin=(0, 0))
        polys = draw.translate(polys, p.pos_x, p.pos_y)
        [rect_jj, pad_top, pad_bot, rect_pk] = polys

        # Use the geometry to create Metal qgeometry
        self.add_qgeometry('poly',
                           dict(pad_top=pad_top, pad_bot=pad_bot),
                           chip=chip)
        self.add_qgeometry('poly',
                           dict(rect_pk=rect_pk),
                           subtract=True,
                           chip=chip)
        # self.add_qgeometry('poly', dict(
        #     rect_jj=rect_jj), helper=True)
        self.add_qgeometry('junction',
                           dict(rect_jj=rect_jj),
                           width=p.inductor_width,
                           chip=chip)

    def make_connection_pads(self):
        """Makes standard transmon in a pocket."""
        for name in self.options.connection_pads:
            self.make_connection_pad(name)

    def make_connection_pad(self, name: str):
        """Makes n individual connector.

        Args:
            name (str) : Name of the connector
        """
        # self.p allows us to directly access parsed values (string -> numbers) form the user option
        p = self.p
        pc = self.p.connection_pads[name]  # parser on connector options
        # extract chip name
        chip = p.chip
    
        if pc.connector_type == 0:  # Claw connector
            
            # define commonly used variables once
            pad_width = p.pad_width
            pad_height = p.pad_height
            pad_gap = p.pad_gap
            pocket_width = p.pocket_width
            pocket_height = p.pocket_height
    
            c_g = pc.claw_gap
            c_l = pc.claw_length
            c_w = pc.claw_width
            c_c_w = pc.claw_cpw_width
            c_c_l = pc.claw_cpw_length
            g_s = pc.ground_spacing
            con_loc = pc.connector_location
    
            claw_cpw = draw.box(-c_w, -c_c_w / 2, -c_c_l - c_w, c_c_w / 2)
            if con_loc == 0 or  con_loc == 2 or  con_loc == 3 or  con_loc == 5:
                t_claw_height = 2*c_g + 2 * c_w + pad_height  # temp value
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, c_g)
            elif con_loc == 1 or  con_loc == 4:
                t_claw_height = 2*c_g + 2 * c_w + pad_width  # temp value
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, g_s)
            else:
                print('Not appropriate position of connector')
            
            port_line = draw.LineString([(- c_w, 0),
                                     (-c_c_l - c_w, 0)])

            claw_rotate = 0
            if con_loc == 0:
                claw_rotate = -180
            elif con_loc == 1:
                claw_rotate = -90
            elif con_loc == 2:
                claw_rotate = 0
            elif con_loc == 3:
                claw_rotate = 0
            elif con_loc == 4:
                claw_rotate = 90
            elif con_loc == 5:
                claw_rotate = 180
    
            claw_translate_x = (con_loc==0 or con_loc==5)*(pad_width/2 + g_s) + (con_loc==2 or con_loc==3)*(-pad_width/2 - g_s)
            claw_translate_y = (con_loc==0 or con_loc==2)*(pad_height + pad_gap)/2 + (con_loc==5 or con_loc==3)*(-pad_height - pad_gap)/2 + (con_loc==1)*(pad_height + pad_gap/2 + g_s) - (con_loc==4)*(pad_height + pad_gap/2 + g_s)
            
            # Rotates and translates the connector polygons (and temporary port_line)
            polys = [connector_arm, connector_etcher, port_line]
            polys = draw.rotate(polys, claw_rotate, origin=(0, 0))
            polys = draw.translate(polys, claw_translate_x, claw_translate_y)
            #polys = draw.rotate(polys, p.orientation, origin=(0, 0))
            polys = draw.translate(polys, p.pos_x, p.pos_y)
            [connector_arm, connector_etcher, port_line] = polys
    
            # Generates qgeometry for the connector pads
            self.add_qgeometry('poly', {f'{name}_connector_arm': connector_arm},
                               chip=chip)
            self.add_qgeometry('poly',
                               {f'{name}_connector_etcher': connector_etcher},
                               subtract=True,
                               chip=chip)
    
            self.add_pin(name, points=port_line.coords, width=c_c_w,input_as_norm=True)
        elif pc.connector_type == 1:  # T-shape connector
            
            # define commonly used variables once
            pad_width = p.pad_width
            pad_height = p.pad_height
            pad_gap = p.pad_gap
            pocket_width = p.pocket_width
            pocket_height = p.pocket_height
    
            c_g = pc.claw_gap
            c_l = 0
            c_w = pc.claw_width
            c_c_w = pc.claw_cpw_width
            c_c_l = pc.claw_cpw_length
            g_s = pc.ground_spacing
            con_loc = pc.connector_location
    
            claw_cpw = draw.box(-c_w, -c_c_w / 2, -c_c_l - c_w, c_c_w / 2)
            if con_loc == 0 or  con_loc == 2 or  con_loc == 3 or  con_loc == 5:
                t_claw_height = pc.t_claw_height
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, c_g)
            elif con_loc == 1 or  con_loc == 4:
                t_claw_height = pc.t_claw_height
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, c_g)
            else:
                print('Not appropriate position of connector')
            
            port_line = draw.LineString([(- c_w, 0),
                                     (-c_c_l - c_w, 0)])

            claw_rotate = 0
            if con_loc == 0:
                claw_rotate = -180
            elif con_loc == 1:
                claw_rotate = -90
            elif con_loc == 2:
                claw_rotate = 0
            elif con_loc == 3:
                claw_rotate = 0
            elif con_loc == 4:
                claw_rotate = 90
            elif con_loc == 5:
                claw_rotate = 180
    
            claw_translate_x = (con_loc==0 or con_loc==5)*(pad_width/2 + g_s) + (con_loc==2 or con_loc==3)*(-pad_width/2 - g_s)
            claw_translate_y = (con_loc==0 or con_loc==2)*(pad_height + pad_gap)/2 + (con_loc==5 or con_loc==3)*(-pad_height - pad_gap)/2 + (con_loc==1)*(pad_height + pad_gap/2 + g_s) - (con_loc==4)*(pad_height + pad_gap/2 + g_s)
            
            # Rotates and translates the connector polygons (and temporary port_line)
            polys = [connector_arm, connector_etcher, port_line]
            polys = draw.rotate(polys, claw_rotate, origin=(0, 0))
            polys = draw.translate(polys, claw_translate_x, claw_translate_y)
            polys = draw.rotate(polys, p.orientation, origin=(0, 0))
            polys = draw.translate(polys, p.pos_x, p.pos_y)
            [connector_arm, connector_etcher, port_line] = polys
    
            # Generates qgeometry for the connector pads
            self.add_qgeometry('poly', {f'{name}_connector_arm': connector_arm},
                               chip=chip)
            self.add_qgeometry('poly',
                               {f'{name}_connector_etcher': connector_etcher},
                               subtract=True,
                               chip=chip)
    
            self.add_pin(name, points=port_line.coords, width=c_c_w,input_as_norm=True)

        else:
            print('Incorrect connector type')
            
        

## Define "LineTee_sqnl"

In [5]:
from qiskit_metal.qlibrary.core import QComponent
class LineTee_sqnl(QComponent):
    """Generates a three pin (+) structure comprised of a primary two pin CPW
    transmission line, and a secondary one pin neighboring CPW transmission
    line that is capacitively coupled to the primary. Such a structure can be
    used, as an example, for generating CPW resonator hangars off of a
    transmission line. (0,0) represents the center position of the component.

    Inherits QComponent class.

    ::

                  (0,0)
        +--------------------------+
                    |
                    |
                    |
                    |
                    |
                    +

    .. image::
        LineTee.png

    .. meta::
        Line Tee

    Options:
        * prime_width: '10um' -- The width of the trace of the two pin CPW transmission line
        * prime_gap: '6um' -- The dielectric gap of the two pin CPW transmission line
        * second_width: '10um' -- The width of the trace of the one pin CPW transmission line
        * second_gap: '6um' -- The dielectric gap of the one pin CPW transmission line (also for the capacitor)
        * t_length: '50um' -- The length for the t branches
        * C_box_width: '50um'
        * C_box_height: '20um'
        * C_box_gap: '20um'
        * C_gap: '5um'
    """
    component_metadata = Dict(short_name='cpw', _qgeometry_table_path='True')
    """Component metadata"""

    #Currently setting the primary CPW length based on the coupling_length
    #May want it to be it's own value that the user can control?
    default_options = Dict(prime_width='10um',
                           prime_gap='6um',
                           second_width='10um',
                           second_gap='6um',
                           t_length='50um')
    """Default connector options"""

    TOOLTIP = """Generates a three pin (+) structure comprised of a primary two pin CPW
    transmission line, and a secondary one pin neighboring CPW transmission
    line that is capacitively coupled to the primary."""

    def make(self):
        """Build the component."""
        p = self.p
        prime_cpw_length = p.t_length * 2
        C_box_width = p.C_box_width
        C_box_height = p.C_box_height
        C_box_gap = p.C_box_gap
        C_gap = p.C_gap
        
        #Primary CPW
        prime_cpw = draw.LineString([[-prime_cpw_length / 2, 0],
                                     [prime_cpw_length / 2, 0]])

        #Secondary CPW
        second_cpw = draw.LineString([[0, -p.prime_width / 2], [0,
                                                                -p.t_length]])
        
        #Capacitor
        C_box = draw.box(-C_box_width/2,-p.t_length-C_box_height,C_box_width/2,-p.t_length)
        C_gap = draw.box(-C_box_width/2-C_box_gap,-p.t_length-C_box_height-C_gap,C_box_width/2+C_box_gap,-p.t_length+C_box_gap)
        
        #Rotate and Translate
        c_items = [prime_cpw, second_cpw]
        c_items = draw.rotate(c_items, p.orientation, origin=(0, 0))
        c_items = draw.translate(c_items, p.pos_x, p.pos_y)
        [prime_cpw, second_cpw] = c_items
        
        C_boxes = [C_box,C_gap]
        C_boxes = draw.rotate(C_boxes, p.orientation, origin=(0, 0))
        C_boxes = draw.translate(C_boxes, p.pos_x, p.pos_y)
        [C_box,C_gap] = C_boxes
        
        #Add to qgeometry tables
        self.add_qgeometry('path', {'prime_cpw': prime_cpw},
                           width=p.prime_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'prime_cpw_sub': prime_cpw},
                           width=p.prime_width + 2 * p.prime_gap,
                           subtract=True,
                           layer=p.layer)
        self.add_qgeometry('path', {'second_cpw': second_cpw},
                           width=p.second_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'second_cpw_sub': second_cpw},
                           width=p.second_width + 2 * p.second_gap,
                           subtract=True,
                           layer=p.layer)
        
        self.add_qgeometry('poly', {'C_box': C_box},
                           layer=p.layer)
        self.add_qgeometry('poly', {'C_gap': C_gap},
                           subtract=True,
                           layer=p.layer)

        #Add pins
        prime_pin_list = prime_cpw.coords
        second_pin_list = second_cpw.coords

        self.add_pin('prime_start',
                     points=np.array(prime_pin_list[::-1]),
                     width=p.prime_width,
                     input_as_norm=True)
        self.add_pin('prime_end',
                     points=np.array(prime_pin_list),
                     width=p.prime_width,
                     input_as_norm=True)
        self.add_pin('second_end',
                     points=np.array(second_pin_list),
                     width=p.second_width,
                     input_as_norm=True)


## Chip dimensions

In [6]:
design = metal.designs.DesignPlanar()

gui = metal.MetalGUI(design)

In [7]:
design.overwrite_enabled = True
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '6mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [8]:
design.chips

{'main': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '9mm',
   'size_y': '6mm',
   'size_z': '-750um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [9]:
design.chips.main.size.size_x = '10mm'
design.chips.main.size.size_y = '10mm'

# Qubits

In [10]:
TransmonPocket_sqnl.get_template_options(design)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'connection_pads': {},
 '_default_connection_pads': {'connector_type': '0',
  'pad_width': '125um',
  'pad_height': '30um',
  'pad_cpw_shift': '0um',
  'pad_cpw_extent': '25um',
  'claw_length': '30um',
  'claw_width': '10um',
  'claw_gap': '6um',
  'claw_cpw_length': '40um',
  'claw_cpw_width': '10um',
  'ground_spacing': '5um',
  'connector_location': '0'},
 'pad_gap': '30um',
 'inductor_width': '20um',
 'pad_width': '455um',
 'pad_height': '90um',
 'pocket_width': '650um',
 'pocket_height': '650um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'aedt_q3d_wire_bonds': False,
 'aedt_hfss_wire_bonds': False,
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '10nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction',
 'aedt_q3d_inductance': 1e-08,
 'aedt_q3d_c

In [11]:
options_1 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    connection_pads=dict(
        readout1 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='30um',
                       claw_width='20um',
                       claw_gap='30um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '0'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        readout2 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='30um',
                       claw_width='20um',
                       claw_gap='30um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '3'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        control = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '30um',
                       ground_spacing='100um',
                       claw_width='30um',
                       claw_gap='18um',
                       claw_cpw_length='100um',
                       claw_cpw_width='30um',
                       connector_location=
                       '1'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) 
    ))

options_2 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    connection_pads=dict(
        readout1 = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='30um',
                       claw_width='20um',
                       claw_gap='30um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '1'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        readout2 = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='30um',
                       claw_width='20um',
                       claw_gap='30um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '4'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        control = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '30um',
                       ground_spacing='100um',
                       claw_width='30um',
                       claw_gap='18um',
                       claw_cpw_length='100um',
                       claw_cpw_width='30um',
                       connector_location=
                       '2'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) 
    ))

options_3 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    connection_pads=dict(
        readout1 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='30um',
                       claw_width='20um',
                       claw_gap='30um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '0'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        readout2 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='30um',
                       claw_width='20um',
                       claw_gap='30um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '3'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        control = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '30um',
                       ground_spacing='100um',
                       claw_width='30um',
                       claw_gap='18um',
                       claw_cpw_length='100um',
                       claw_cpw_width='30um',
                       connector_location=
                       '4'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) 
    ))

options_4 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    connection_pads=dict(
        readout1 = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='30um',
                       claw_width='20um',
                       claw_gap='30um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '1'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        readout2 = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='30um',
                       claw_width='20um',
                       claw_gap='30um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '4'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        control = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '30um',
                       ground_spacing='100um',
                       claw_width='30um',
                       claw_gap='18um',
                       claw_cpw_length='100um',
                       claw_cpw_width='30um',
                       connector_location=
                       '5'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) 
    ))


In [12]:
q_1 = TransmonPocket_sqnl(design,'Q_1', options = dict(
        pos_x='0mm', 
        pos_y='2.5mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options_1))
q_2 = TransmonPocket_sqnl(design,'Q_2', options = dict(
        pos_x='-2.5mm', 
        pos_y='0mm', 
        gds_cell_name ='FakeJunction_02',
        hfss_inductance ='14nH',
        **options_2))
q_3 = TransmonPocket_sqnl(design,'Q_3', options = dict(
        pos_x='0mm', 
        pos_y='-2.5mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options_3))
q_4 = TransmonPocket_sqnl(design,'Q_4', options = dict(
        pos_x='2.5mm', 
        pos_y='0mm', 
        gds_cell_name ='FakeJunction_02',
        hfss_inductance ='14nH',
        **options_4))
gui.rebuild()
gui.autoscale()

# Resoantors

## Launch Pads

In [13]:
launch_TL_read_11 = LaunchpadWirebond(design, 
                                   'launch_TL_read_11', 
                                   options = dict(
                                       pos_x = '2.5mm', 
                                       pos_y ='4.3mm', 
                                       orientation = '270',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_12 = LaunchpadWirebond(design, 
                                   'launch_TL_read_12', 
                                   options = dict(
                                       pos_x = '0mm', 
                                       pos_y ='4.3mm', 
                                       orientation = '270',
                                       trace_gap = '18um',
                                       trace_width = '30um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_13 = LaunchpadWirebond(design, 
                                   'launch_TL_read_13', 
                                   options = dict(
                                       pos_x = '-2.5mm', 
                                       pos_y ='4.3mm', 
                                       orientation = '270',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_21 = LaunchpadWirebond(design, 
                                   'launch_TL_read_21', 
                                   options = dict(
                                       pos_x = '-4.3mm', 
                                       pos_y ='2.5mm', 
                                       orientation = '0',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_22 = LaunchpadWirebond(design, 
                                   'launch_TL_read_22', 
                                   options = dict(
                                       pos_x = '-4.3mm', 
                                       pos_y ='0mm', 
                                       orientation = '0',
                                       trace_gap = '18um',
                                       trace_width = '30um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_23 = LaunchpadWirebond(design, 
                                   'launch_TL_read_23', 
                                   options = dict(
                                       pos_x = '-4.3mm', 
                                       pos_y ='-2.5mm', 
                                       orientation = '0',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_31 = LaunchpadWirebond(design, 
                                   'launch_TL_read_31', 
                                   options = dict(
                                       pos_x = '-2.5mm', 
                                       pos_y ='-4.3mm', 
                                       orientation = '90',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_32 = LaunchpadWirebond(design, 
                                   'launch_TL_read_32', 
                                   options = dict(
                                       pos_x = '0mm', 
                                       pos_y ='-4.3mm', 
                                       orientation = '90',
                                       trace_gap = '18um',
                                       trace_width = '30um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_33 = LaunchpadWirebond(design, 
                                   'launch_TL_read_33', 
                                   options = dict(
                                       pos_x = '2.5mm', 
                                       pos_y ='-4.3mm', 
                                       orientation = '90',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_41 = LaunchpadWirebond(design, 
                                   'launch_TL_read_41', 
                                   options = dict(
                                       pos_x = '4.3mm', 
                                       pos_y ='-2.5mm', 
                                       orientation = '180',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_42 = LaunchpadWirebond(design, 
                                   'launch_TL_read_42', 
                                   options = dict(
                                       pos_x = '4.3mm', 
                                       pos_y ='0mm', 
                                       orientation = '180',
                                       trace_gap = '18um',
                                       trace_width = '30um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_43 = LaunchpadWirebond(design, 
                                   'launch_TL_read_43', 
                                   options = dict(
                                       pos_x = '4.3mm', 
                                       pos_y ='2.5mm', 
                                       orientation = '180',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))


gui.rebuild()
gui.autoscale()

In [14]:
CapNInterdigital.get_template_options(design)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'north_width': '10um',
 'north_gap': '6um',
 'south_width': '10um',
 'south_gap': '6um',
 'cap_width': '10um',
 'cap_gap': '6um',
 'cap_gap_ground': '6um',
 'finger_length': '20um',
 'finger_count': '5',
 'cap_distance': '50um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'aedt_q3d_wire_bonds': False,
 'aedt_hfss_wire_bonds': False}

## IDCs

### Definition

In [15]:
from qiskit_metal.qlibrary.core import QComponent
class CapNInterdigital_sqnl(QComponent):
    
    component_metadata = Dict(short_name='cpw',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_path='True')
    """Component metadata"""

    #Currently setting the primary CPW length based on the coupling_length
    #May want it to be it's own value that the user can control?
    default_options = Dict(north_width='10um',
                           north_gap='6um',
                           south_width='10um',
                           south_gap='6um',
                           cap_width='10um',
                           cap_gap='6um',
                           cap_gap_ground='6um',
                           finger_length='20um',
                           finger_count='5',
                           cap_distance='50um',
                           taper_length='200um'
                          )
    """Default connector options"""

    TOOLTIP = """Generates a two pin (+) structure
     comprised of a north CPW transmission line, 
     and a south transmission line, coupled 
     together via a finger capacitor."""

    def make(self):
        """Build the component."""
        p = self.p
        N = int(p.finger_count)

        #Finger Capacitor
        cap_box = draw.Polygon([[(N * p.cap_width + (N - 1) * p.cap_gap)/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [p.north_width/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-p.north_width/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-p.south_width/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [p.south_width/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [(N * p.cap_width + (N - 1) * p.cap_gap)/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2]
                               ])
        make_cut_list = []
        make_cut_list.append([0, (p.finger_length) / 2])
        make_cut_list.append([(p.cap_width) + (p.cap_gap / 2),
                              (p.finger_length) / 2])
        flip = -1

        for i in range(1, N):
            make_cut_list.append([
                i * (p.cap_width) + (2 * i - 1) * (p.cap_gap / 2),
                flip * (p.finger_length) / 2
            ])
            make_cut_list.append([
                (i + 1) * (p.cap_width) + (2 * i + 1) * (p.cap_gap / 2),
                flip * (p.finger_length) / 2
            ])
            flip = flip * -1

        cap_cut = draw.LineString(make_cut_list).buffer(p.cap_gap / 2,
                                                        cap_style=2,
                                                        join_style=2)
        cap_cut = draw.translate(cap_cut,
                                 -(N * p.cap_width + (N - 1) * p.cap_gap) / 2,
                                 0)

        cap_body = draw.subtract(cap_box, cap_cut)
        cap_body = draw.translate(
            cap_body, 0, -p.cap_distance -
            (p.cap_gap + 2 * p.cap_width + p.finger_length) / 2)

        cap_etch = draw.Polygon([[(N * p.cap_width + (N - 1) * p.cap_gap)/2+p.cap_gap_ground, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [p.north_width/2+p.north_gap, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-p.north_width/2-p.north_gap, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2-p.cap_gap_ground, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2-p.cap_gap_ground, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-p.south_width/2-p.south_gap, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [p.south_width/2+p.south_gap, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [(N * p.cap_width + (N - 1) * p.cap_gap)/2+p.cap_gap_ground, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2]
                               ])

        cap_etch = draw.translate(
            cap_etch, 0, -p.cap_distance -
            (p.cap_gap + 2 * p.cap_width + p.finger_length) / 2)

        #CPW
        north_cpw = draw.LineString([[0, p.taper_length], [0, p.taper_length-p.cap_distance]])

        south_cpw = draw.LineString(
            [[
                0, -p.taper_length-p.cap_distance -
                (p.cap_gap + 2 * p.cap_width + p.finger_length)
            ],
             [
                 0, -p.taper_length-2 * p.cap_distance -
                 (p.cap_gap + 2 * p.cap_width + p.finger_length)
             ]])

        #Rotate and Translate
        c_items = [north_cpw, south_cpw, cap_body, cap_etch]
        c_items = draw.rotate(c_items, p.orientation, origin=(0, 0))
        c_items = draw.translate(c_items, p.pos_x, p.pos_y)
        [north_cpw, south_cpw, cap_body, cap_etch] = c_items

        #Add to qgeometry tables
        self.add_qgeometry('path', {'north_cpw': north_cpw},
                           width=p.north_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'north_cpw_sub': north_cpw},
                           width=p.north_width + 2 * p.north_gap,
                           layer=p.layer,
                           subtract=True)

        self.add_qgeometry('path', {'south_cpw': south_cpw},
                           width=p.south_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'south_cpw_sub': south_cpw},
                           width=p.south_width + 2 * p.south_gap,
                           layer=p.layer,
                           subtract=True)

        self.add_qgeometry('poly', {'cap_body': cap_body}, layer=p.layer)
        self.add_qgeometry('poly', {'cap_etch': cap_etch},
                           layer=p.layer,
                           subtract=True)

        #Add pins
        north_pin_list = north_cpw.coords
        south_pin_list = south_cpw.coords

        self.add_pin('north_end',
                     points=np.array(north_pin_list[::-1]),
                     width=p.north_width,
                     input_as_norm=True)
        self.add_pin('south_end',
                     points=np.array(south_pin_list),
                     width=p.south_width,
                     input_as_norm=True)

### Draw

In [16]:
highC_PF_TL_11 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_11', 
                               options = dict(
                                   pos_x = '2.5mm',
                                   pos_y = '3.9mm',
                                   orientation = '0',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))
highC_PF_TL_13 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_13', 
                               options = dict(
                                   pos_x = '-2.5mm',
                                   pos_y = '3.9mm',
                                   orientation = '0',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))
highC_PF_TL_21 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_21', 
                               options = dict(
                                   pos_x = '-3.9mm',
                                   pos_y = '2.5mm',
                                   orientation = '90',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))
highC_PF_TL_23 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_23', 
                               options = dict(
                                   pos_x = '-3.9mm',
                                   pos_y = '-2.5mm',
                                   orientation = '90',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))
highC_PF_TL_31 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_31', 
                               options = dict(
                                   pos_x = '-2.5mm',
                                   pos_y = '-3.9mm',
                                   orientation = '180',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))
highC_PF_TL_33 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_33', 
                               options = dict(
                                   pos_x = '2.5mm',
                                   pos_y = '-3.9mm',
                                   orientation = '180',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))
highC_PF_TL_41 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_41', 
                               options = dict(
                                   pos_x = '3.9mm',
                                   pos_y = '-2.5mm',
                                   orientation = '270',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))
highC_PF_TL_43 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_43', 
                               options = dict(
                                   pos_x = '3.9mm',
                                   pos_y = '2.5mm',
                                   orientation = '270',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))

gui.rebuild()
gui.autoscale()

In [17]:
option_TL_11 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_11',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_11',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )
option_TL_13 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_13',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_13',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )
option_TL_21 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_21',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_21',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )
option_TL_23 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_23',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_23',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )
option_TL_31 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_31',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_31',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )
option_TL_33 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_33',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_33',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )
option_TL_41 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_41',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_41',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )
option_TL_43 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_43',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_43',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )

TL_11 = RouteStraight(design, 'TL_11', options=option_TL_11)
TL_13 = RouteStraight(design, 'TL_13', options=option_TL_13)
TL_21 = RouteStraight(design, 'TL_21', options=option_TL_21)
TL_23 = RouteStraight(design, 'TL_23', options=option_TL_23)
TL_31 = RouteStraight(design, 'TL_31', options=option_TL_31)
TL_33 = RouteStraight(design, 'TL_33', options=option_TL_33)
TL_41 = RouteStraight(design, 'TL_41', options=option_TL_41)
TL_43 = RouteStraight(design, 'TL_43', options=option_TL_43)

gui.rebuild()
gui.autoscale()

## Purcell filters

### PF_11

In [18]:
pin_opt_PF_11 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_11',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_11_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )


stg_PF_11 = ShortToGround(design, 'PF_11_short', 
                       options=dict(
                           pos_x='2.30mm', 
                           pos_y='3.0mm',
                           width='20um',
                           gap='10um',
                           orientation=0))

pin_opt_PF_11.pin_inputs.start_pin.component = 'highC_PF_TL_11'
pin_opt_PF_11.pin_inputs.end_pin.component = 'PF_11_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_11.lead.start_straight = '100um + 90um'
pin_opt_PF_11.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_11 = OrderedDict()
jogsS_11[0] = ["R", '700um + 180um']
jogsS_11[1] = ["L", '700um + 180um']
jogsS_11[2] = ["L", '0um + 180um']
jogsS_11[3] = ["L", '0um + 180um']
jogsS_11[4] = ["R", '0um + 180um']
jogsS_11[5] = ["L", '0um + 180um']
jogsS_11[6] = ["L", '0um + 180um']
jogsS_11[7] = ["R", '0um + 180um']

pin_opt_PF_11.lead.start_jogged_extension = jogsS_11
PF_11 = RouteFramed(design, 'PF_11', pin_opt_PF_11)

# gui.rebuild()
# gui.autoscale()

In [19]:
# PF-R coupler 11
coupler_length_11 = 98e-3

T_coupler_PF_11 = LineTee_sqnl(design, 'T_coupler_PF_11', options=dict(pos_x='1.62mm',
                                             pos_y='2.94mm',
                                             t_length=coupler_length_11,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = -90
                                             ))

# gui.rebuild()
# gui.autoscale()

### PF_13

In [20]:
pin_opt_PF_13 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_13',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_13_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )


stg_PF_13 = ShortToGround(design, 'PF_13_short', 
                       options=dict(
                           pos_x='-2.30mm', 
                           pos_y='3.0mm',
                           width='20um',
                           gap='10um',
                           orientation=180))

pin_opt_PF_13.pin_inputs.start_pin.component = 'highC_PF_TL_13'
pin_opt_PF_13.pin_inputs.end_pin.component = 'PF_13_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_13.lead.start_straight = '100um + 90um'
pin_opt_PF_13.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_13 = OrderedDict()
jogsS_13[0] = ["L", '700um + 180um']
jogsS_13[1] = ["R", '700um + 180um']
jogsS_13[2] = ["R", '0um + 180um']
jogsS_13[3] = ["R", '0um + 180um']
jogsS_13[4] = ["L", '0um + 180um']
jogsS_13[5] = ["R", '0um + 180um']
jogsS_13[6] = ["R", '0um + 180um']
jogsS_13[7] = ["L", '0um + 180um']

pin_opt_PF_13.lead.start_jogged_extension = jogsS_13
PF_13 = RouteFramed(design, 'PF_13', pin_opt_PF_13)

# gui.rebuild()
# gui.autoscale()

In [21]:
# PF-R coupler 13
coupler_length_13 = 80e-3

T_coupler_PF_13 = LineTee_sqnl(design, 'T_coupler_PF_13', options=dict(pos_x='-1.62mm',
                                             pos_y='2.74mm',
                                             t_length=coupler_length_13,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 90
                                             ))

# gui.rebuild()
# gui.autoscale()

### PF_21

In [22]:
pin_opt_PF_21 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_21',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_21_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )


stg_PF_21 = ShortToGround(design, 'PF_21_short', 
                       options=dict(
                           pos_x='-3.0mm', 
                           pos_y='2.30mm',
                           width='20um',
                           gap='10um',
                           orientation=90))

pin_opt_PF_21.pin_inputs.start_pin.component = 'highC_PF_TL_21'
pin_opt_PF_21.pin_inputs.end_pin.component = 'PF_21_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_21.lead.start_straight = '100um + 90um'
pin_opt_PF_21.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_21 = OrderedDict()
jogsS_21[0] = ["R", '700um + 180um']
jogsS_21[1] = ["L", '700um + 180um']
jogsS_21[2] = ["L", '0um + 180um']
jogsS_21[3] = ["L", '0um + 180um']
jogsS_21[4] = ["R", '0um + 180um']
jogsS_21[5] = ["L", '0um + 180um']
jogsS_21[6] = ["L", '0um + 180um']
jogsS_21[7] = ["R", '0um + 180um']

pin_opt_PF_21.lead.start_jogged_extension = jogsS_21
PF_21 = RouteFramed(design, 'PF_21', pin_opt_PF_21)

# gui.rebuild()
# gui.autoscale()

In [23]:
# PF-R coupler 21
coupler_length_21 = 80e-3

T_coupler_PF_21 = LineTee_sqnl(design, 'T_coupler_PF_21', options=dict(pos_x='-2.74mm',
                                             pos_y='1.62mm',
                                             t_length=coupler_length_21,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 0
                                             ))

# gui.rebuild()
# gui.autoscale()

### PF_23

In [24]:
pin_opt_PF_23 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_23',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_23_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )


stg_PF_23 = ShortToGround(design, 'PF_23_short', 
                       options=dict(
                           pos_x='-3.00mm', 
                           pos_y='-2.30mm',
                           width='20um',
                           gap='10um',
                           orientation=270))

pin_opt_PF_23.pin_inputs.start_pin.component = 'highC_PF_TL_23'
pin_opt_PF_23.pin_inputs.end_pin.component = 'PF_23_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_23.lead.start_straight = '100um + 90um'
pin_opt_PF_23.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_23 = OrderedDict()
jogsS_23[0] = ["L", '700um + 180um']
jogsS_23[1] = ["R", '700um + 180um']
jogsS_23[2] = ["R", '0um + 180um']
jogsS_23[3] = ["R", '0um + 180um']
jogsS_23[4] = ["L", '0um + 180um']
jogsS_23[5] = ["R", '0um + 180um']
jogsS_23[6] = ["R", '0um + 180um']
jogsS_23[7] = ["L", '0um + 180um']

pin_opt_PF_23.lead.start_jogged_extension = jogsS_23
PF_23 = RouteFramed(design, 'PF_23', pin_opt_PF_23)

# gui.rebuild()
# gui.autoscale()

In [25]:
# PF-R coupler 23
coupler_length_23 = 80e-3

T_coupler_PF_23 = LineTee_sqnl(design, 'T_coupler_PF_23', options=dict(pos_x='-2.74mm',
                                             pos_y='-1.62mm',
                                             t_length=coupler_length_23,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 180
                                             ))

# gui.rebuild()
# gui.autoscale()

### PF_31

In [26]:
pin_opt_PF_31 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_31',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_31_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )


stg_PF_31 = ShortToGround(design, 'PF_31_short', 
                       options=dict(
                           pos_x='-2.3mm', 
                           pos_y='-3.0mm',
                           width='20um',
                           gap='10um',
                           orientation=180))

pin_opt_PF_31.pin_inputs.start_pin.component = 'highC_PF_TL_31'
pin_opt_PF_31.pin_inputs.end_pin.component = 'PF_31_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_31.lead.start_straight = '100um + 90um'
pin_opt_PF_31.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_31 = OrderedDict()
jogsS_31[0] = ["R", '700um + 180um']
jogsS_31[1] = ["L", '700um + 180um']
jogsS_31[2] = ["L", '0um + 180um']
jogsS_31[3] = ["L", '0um + 180um']
jogsS_31[4] = ["R", '0um + 180um']
jogsS_31[5] = ["L", '0um + 180um']
jogsS_31[6] = ["L", '0um + 180um']
jogsS_31[7] = ["R", '0um + 180um']

pin_opt_PF_31.lead.start_jogged_extension = jogsS_31
PF_31 = RouteFramed(design, 'PF_31', pin_opt_PF_31)

# gui.rebuild()
# gui.autoscale()

In [27]:
# PF-R coupler 31
coupler_length_31 = 80e-3

T_coupler_PF_31 = LineTee_sqnl(design, 'T_coupler_PF_31', options=dict(pos_x='-1.62mm',
                                             pos_y='-2.74mm',
                                             t_length=coupler_length_31,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 90
                                             ))

# gui.rebuild()
# gui.autoscale()

### PF_33

In [28]:
pin_opt_PF_33 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_33',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_33_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )


stg_PF_33 = ShortToGround(design, 'PF_33_short', 
                       options=dict(
                           pos_x='2.30mm', 
                           pos_y='-3.0mm',
                           width='20um',
                           gap='10um',
                           orientation=0))

pin_opt_PF_33.pin_inputs.start_pin.component = 'highC_PF_TL_33'
pin_opt_PF_33.pin_inputs.end_pin.component = 'PF_33_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_33.lead.start_straight = '100um + 90um'
pin_opt_PF_33.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_33 = OrderedDict()
jogsS_33[0] = ["L", '700um + 180um']
jogsS_33[1] = ["R", '700um + 180um']
jogsS_33[2] = ["R", '0um + 180um']
jogsS_33[3] = ["R", '0um + 180um']
jogsS_33[4] = ["L", '0um + 180um']
jogsS_33[5] = ["R", '0um + 180um']
jogsS_33[6] = ["R", '0um + 180um']
jogsS_33[7] = ["L", '0um + 180um']

pin_opt_PF_33.lead.start_jogged_extension = jogsS_33
PF_33 = RouteFramed(design, 'PF_33', pin_opt_PF_33)

# gui.rebuild()
# gui.autoscale()

In [29]:
# PF-R coupler 33
coupler_length_33 = 80e-3

T_coupler_PF_33 = LineTee_sqnl(design, 'T_coupler_PF_33', options=dict(pos_x='1.62mm',
                                             pos_y='-2.74mm',
                                             t_length=coupler_length_33,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 270
                                             ))

# gui.rebuild()
# gui.autoscale()

### PF_41

In [30]:
pin_opt_PF_41 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_41',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_41_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )


stg_PF_41 = ShortToGround(design, 'PF_41_short', 
                       options=dict(
                           pos_x='3.0mm', 
                           pos_y='-2.30mm',
                           width='20um',
                           gap='10um',
                           orientation=270))

pin_opt_PF_41.pin_inputs.start_pin.component = 'highC_PF_TL_41'
pin_opt_PF_41.pin_inputs.end_pin.component = 'PF_41_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_41.lead.start_straight = '100um + 90um'
pin_opt_PF_41.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_41 = OrderedDict()
jogsS_41[0] = ["R", '700um + 180um']
jogsS_41[1] = ["L", '700um + 180um']
jogsS_41[2] = ["L", '0um + 180um']
jogsS_41[3] = ["L", '0um + 180um']
jogsS_41[4] = ["R", '0um + 180um']
jogsS_41[5] = ["L", '0um + 180um']
jogsS_41[6] = ["L", '0um + 180um']
jogsS_41[7] = ["R", '0um + 180um']

pin_opt_PF_41.lead.start_jogged_extension = jogsS_41
PF_41 = RouteFramed(design, 'PF_41', pin_opt_PF_41)

# gui.rebuild()
# gui.autoscale()

In [31]:
# PF-R coupler 41
coupler_length_41 = 80e-3

T_coupler_PF_41 = LineTee_sqnl(design, 'T_coupler_PF_41', options=dict(pos_x='2.74mm',
                                             pos_y='-1.62mm',
                                             t_length=coupler_length_41,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 180
                                             ))

# gui.rebuild()
# gui.autoscale()

### PF_43

In [32]:
pin_opt_PF_43 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_43',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_43_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )


stg_PF_43 = ShortToGround(design, 'PF_43_short', 
                       options=dict(
                           pos_x='3.00mm', 
                           pos_y='2.30mm',
                           width='20um',
                           gap='10um',
                           orientation=90))

pin_opt_PF_43.pin_inputs.start_pin.component = 'highC_PF_TL_43'
pin_opt_PF_43.pin_inputs.end_pin.component = 'PF_43_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_43.lead.start_straight = '100um + 90um'
pin_opt_PF_43.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_43 = OrderedDict()
jogsS_43[0] = ["L", '700um + 180um']
jogsS_43[1] = ["R", '700um + 180um']
jogsS_43[2] = ["R", '0um + 180um']
jogsS_43[3] = ["R", '0um + 180um']
jogsS_43[4] = ["L", '0um + 180um']
jogsS_43[5] = ["R", '0um + 180um']
jogsS_43[6] = ["R", '0um + 180um']
jogsS_43[7] = ["L", '0um + 180um']

pin_opt_PF_43.lead.start_jogged_extension = jogsS_43
PF_43 = RouteFramed(design, 'PF_43', pin_opt_PF_43)

# gui.rebuild()
# gui.autoscale()

In [33]:
# PF-R coupler 43
coupler_length_43 = 80e-3

T_coupler_PF_43 = LineTee_sqnl(design, 'T_coupler_PF_43', options=dict(pos_x='2.74mm',
                                             pos_y='1.62mm',
                                             t_length=coupler_length_43,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 0
                                             ))

gui.rebuild()
gui.autoscale()

## Readout resonators

### R_11

In [34]:
pin_opt_R_11 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_1',
                                    pin='readout1'),
                                end_pin=Dict(
                                    component='R_11_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

stg_R_11 = ShortToGround(design, 'R_11_short', 
                       options=dict(
                           pos_x='1.01mm', 
                           pos_y='2.8mm',
                           width='20um',
                           gap='10um',
                           orientation=-90))

pin_opt_R_11.pin_inputs.start_pin.component = 'Q_1'
pin_opt_R_11.pin_inputs.end_pin.component = 'R_11_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_11.lead.start_straight = '700um + 90um'
pin_opt_R_11.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_11 = OrderedDict()
jogsS_11[0] = ["R", '0um + 180um']
jogsS_11[1] = ["R", '0um + 180um']
jogsS_11[2] = ["L", '0um + 180um']
jogsS_11[3] = ["L", '0um + 180um']
jogsS_11[4] = ["R", '0um + 180um']
jogsS_11[5] = ["L", '0um + 180um']
jogsS_11[6] = ["L", '1330um + 180um']
jogsS_11[7] = ["L", '0um + 180um']

pin_opt_R_11.lead.start_jogged_extension = jogsS_11
R_11 = RouteFramed(design, 'R_11', pin_opt_R_11)

# gui.rebuild()
# gui.autoscale()

In [35]:
# R-PF coupler 11
coupler_length_11 = 90e-3

T_coupler_R_11 = LineTee_sqnl(design, 'T_coupler_R_11', options=dict(pos_x='1.39mm',
                                             pos_y='2.940mm',
                                             t_length=coupler_length_11,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 90
                                             ))

# gui.rebuild()
# gui.autoscale()

### R_13

In [36]:
pin_opt_R_13 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_1',
                                    pin='readout2'),
                                end_pin=Dict(
                                    component='R_13_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )

stg_R_13 = ShortToGround(design, 'R_13_short', 
                       options=dict(
                           pos_x='-1.01mm', 
                           pos_y='2.8mm',
                           width='20um',
                           gap='10um',
                           orientation=-90))

pin_opt_R_13.pin_inputs.start_pin.component = 'Q_1'
pin_opt_R_13.pin_inputs.end_pin.component = 'R_13_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_13.lead.start_straight = '700um + 90um'
pin_opt_R_13.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_13 = OrderedDict()
jogsS_13[0] = ["L", '0um + 180um']
jogsS_13[1] = ["L", '0um + 180um']
jogsS_13[2] = ["R", '0um + 180um']
jogsS_13[3] = ["R", '0um + 180um']
jogsS_13[4] = ["L", '0um + 180um']
jogsS_13[5] = ["R", '0um + 180um']
jogsS_13[6] = ["R", '1200um + 180um']
jogsS_13[7] = ["R", '0um + 180um']

pin_opt_R_13.lead.start_jogged_extension = jogsS_13
R_13 = RouteFramed(design, 'R_13', pin_opt_R_13)

# gui.rebuild()
# gui.autoscale()

In [37]:
# R-PF coupler 13
coupler_length_13 = 95e-3

T_coupler_R_13 = LineTee_sqnl(design, 'T_coupler_R_13', options=dict(pos_x='-1.39mm',
                                             pos_y='2.740mm',
                                             t_length=coupler_length_13,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = -90
                                             ))

# gui.rebuild()
# gui.autoscale()

### R_21

In [38]:
pin_opt_R_21 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_2',
                                    pin='readout1'),
                                end_pin=Dict(
                                    component='R_21_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )

stg_R_21 = ShortToGround(design, 'R_21_short', 
                       options=dict(
                           pos_x='-2.8mm', 
                           pos_y='1.01mm',
                           width='20um',
                           gap='10um',
                           orientation=0))

pin_opt_R_21.pin_inputs.start_pin.component = 'Q_2'
pin_opt_R_21.pin_inputs.end_pin.component = 'R_21_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_21.lead.start_straight = '805um + 90um'
pin_opt_R_21.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_21 = OrderedDict()
jogsS_21[0] = ["R", '0um + 180um']
jogsS_21[1] = ["R", '0um + 180um']
jogsS_21[2] = ["L", '0um + 180um']
jogsS_21[3] = ["L", '0um + 180um']
jogsS_21[4] = ["R", '0um + 180um']
jogsS_21[5] = ["L", '0um + 180um']
jogsS_21[6] = ["L", '1000um + 180um']
jogsS_21[7] = ["L", '0um + 180um']

pin_opt_R_21.lead.start_jogged_extension = jogsS_21
R_21 = RouteFramed(design, 'R_21', pin_opt_R_21)

# gui.rebuild()
# gui.autoscale()

In [39]:
# R-PF coupler 21
coupler_length_21 = 95e-3

T_coupler_R_21 = LineTee_sqnl(design, 'T_coupler_R_21', options=dict(pos_x='-2.740mm',
                                             pos_y='1.39mm',
                                             t_length=coupler_length_21,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 180
                                             ))

# gui.rebuild()
# gui.autoscale()

### R_23

In [40]:
pin_opt_R_23 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_2',
                                    pin='readout2'),
                                end_pin=Dict(
                                    component='R_23_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )

stg_R_23 = ShortToGround(design, 'R_23_short', 
                       options=dict(
                           pos_x='-2.8mm', 
                           pos_y='-1.01mm',
                           width='20um',
                           gap='10um',
                           orientation=0))

pin_opt_R_23.pin_inputs.start_pin.component = 'Q_2'
pin_opt_R_23.pin_inputs.end_pin.component = 'R_23_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_23.lead.start_straight = '805um + 90um'
pin_opt_R_23.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_23 = OrderedDict()
jogsS_23[0] = ["L", '0um + 180um']
jogsS_23[1] = ["L", '0um + 180um']
jogsS_23[2] = ["R", '0um + 180um']
jogsS_23[3] = ["R", '0um + 180um']
jogsS_23[4] = ["L", '0um + 180um']
jogsS_23[5] = ["R", '0um + 180um']
jogsS_23[6] = ["R", '1200um + 180um']
jogsS_23[7] = ["R", '0um + 180um']

pin_opt_R_23.lead.start_jogged_extension = jogsS_23
R_23 = RouteFramed(design, 'R_23', pin_opt_R_23)

# gui.rebuild()
# gui.autoscale()

In [41]:
# R-PF coupler 23
coupler_length_23 = 95e-3

T_coupler_R_23 = LineTee_sqnl(design, 'T_coupler_R_23', options=dict(pos_x='-2.740mm',
                                             pos_y='-1.39mm',
                                             t_length=coupler_length_23,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 0
                                             ))

# gui.rebuild()
# gui.autoscale()

### R_31

In [42]:
pin_opt_R_31 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_3',
                                    pin='readout2'),
                                end_pin=Dict(
                                    component='R_31_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )

stg_R_31 = ShortToGround(design, 'R_31_short', 
                       options=dict(
                           pos_x='-1.01mm', 
                           pos_y='-2.8mm',
                           width='20um',
                           gap='10um',
                           orientation=90))

pin_opt_R_31.pin_inputs.start_pin.component = 'Q_3'
pin_opt_R_31.pin_inputs.end_pin.component = 'R_31_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_31.lead.start_straight = '700um + 90um'
pin_opt_R_31.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_31 = OrderedDict()
jogsS_31[0] = ["R", '0um + 180um']
jogsS_31[1] = ["R", '0um + 180um']
jogsS_31[2] = ["L", '0um + 180um']
jogsS_31[3] = ["L", '0um + 180um']
jogsS_31[4] = ["R", '0um + 180um']
jogsS_31[5] = ["L", '0um + 180um']
jogsS_31[6] = ["L", '1000um + 180um']
jogsS_31[7] = ["L", '0um + 180um']

pin_opt_R_31.lead.start_jogged_extension = jogsS_31
R_31 = RouteFramed(design, 'R_31', pin_opt_R_31)

# gui.rebuild()
# gui.autoscale()

In [43]:
# R-PF coupler 31
coupler_length_31 = 90e-3

T_coupler_R_31 = LineTee_sqnl(design, 'T_coupler_R_31', options=dict(pos_x='-1.39mm',
                                             pos_y='-2.740mm',
                                             t_length=coupler_length_31,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = -90
                                             ))

# gui.rebuild()
# gui.autoscale()

### R_33

In [44]:
pin_opt_R_33 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_3',
                                    pin='readout1'),
                                end_pin=Dict(
                                    component='R_33_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )

stg_R_33 = ShortToGround(design, 'R_33_short', 
                       options=dict(
                           pos_x='1.01mm', 
                           pos_y='-2.8mm',
                           width='20um',
                           gap='10um',
                           orientation=90))

pin_opt_R_33.pin_inputs.start_pin.component = 'Q_3'
pin_opt_R_33.pin_inputs.end_pin.component = 'R_33_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_33.lead.start_straight = '700um + 90um'
pin_opt_R_33.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_33 = OrderedDict()
jogsS_33[0] = ["L", '0um + 180um']
jogsS_33[1] = ["L", '0um + 180um']
jogsS_33[2] = ["R", '0um + 180um']
jogsS_33[3] = ["R", '0um + 180um']
jogsS_33[4] = ["L", '0um + 180um']
jogsS_33[5] = ["R", '0um + 180um']
jogsS_33[6] = ["R", '1200um + 180um']
jogsS_33[7] = ["R", '0um + 180um']

pin_opt_R_33.lead.start_jogged_extension = jogsS_33
R_33 = RouteFramed(design, 'R_33', pin_opt_R_33)

# gui.rebuild()
# gui.autoscale()

In [45]:
# R-PF coupler 33
coupler_length_33 = 95e-3

T_coupler_R_33 = LineTee_sqnl(design, 'T_coupler_R_33', options=dict(pos_x='1.39mm',
                                             pos_y='-2.740mm',
                                             t_length=coupler_length_13,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 90
                                             ))

# gui.rebuild()
# gui.autoscale()

### R_41

In [46]:
pin_opt_R_41 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_4',
                                    pin='readout2'),
                                end_pin=Dict(
                                    component='R_41_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )

stg_R_41 = ShortToGround(design, 'R_41_short', 
                       options=dict(
                           pos_x='2.8mm', 
                           pos_y='-1.01mm',
                           width='20um',
                           gap='10um',
                           orientation=180))

pin_opt_R_41.pin_inputs.start_pin.component = 'Q_4'
pin_opt_R_41.pin_inputs.end_pin.component = 'R_41_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_41.lead.start_straight = '805um + 90um'
pin_opt_R_41.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_41 = OrderedDict()
jogsS_41[0] = ["R", '0um + 180um']
jogsS_41[1] = ["R", '0um + 180um']
jogsS_41[2] = ["L", '0um + 180um']
jogsS_41[3] = ["L", '0um + 180um']
jogsS_41[4] = ["R", '0um + 180um']
jogsS_41[5] = ["L", '0um + 180um']
jogsS_41[6] = ["L", '1000um + 180um']
jogsS_41[7] = ["L", '0um + 180um']

pin_opt_R_41.lead.start_jogged_extension = jogsS_41
R_41 = RouteFramed(design, 'R_41', pin_opt_R_41)

# gui.rebuild()
# gui.autoscale()

In [47]:
# R-PF coupler 41
coupler_length_41 = 95e-3

T_coupler_R_41 = LineTee_sqnl(design, 'T_coupler_R_41', options=dict(pos_x='2.740mm',
                                             pos_y='-1.39mm',
                                             t_length=coupler_length_41,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 0
                                             ))

# gui.rebuild()
# gui.autoscale()

### R_43

In [48]:
pin_opt_R_43 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_4',
                                    pin='readout1'),
                                end_pin=Dict(
                                    component='R_43_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True
              )

stg_R_43 = ShortToGround(design, 'R_43_short', 
                       options=dict(
                           pos_x='2.8mm', 
                           pos_y='1.01mm',
                           width='20um',
                           gap='10um',
                           orientation=180))

pin_opt_R_43.pin_inputs.start_pin.component = 'Q_4'
pin_opt_R_43.pin_inputs.end_pin.component = 'R_43_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_43.lead.start_straight = '805um + 90um'
pin_opt_R_43.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_43 = OrderedDict()
jogsS_43[0] = ["L", '0um + 180um']
jogsS_43[1] = ["L", '0um + 180um']
jogsS_43[2] = ["R", '0um + 180um']
jogsS_43[3] = ["R", '0um + 180um']
jogsS_43[4] = ["L", '0um + 180um']
jogsS_43[5] = ["R", '0um + 180um']
jogsS_43[6] = ["R", '1200um + 180um']
jogsS_43[7] = ["R", '0um + 180um']

pin_opt_R_43.lead.start_jogged_extension = jogsS_43
R_43 = RouteFramed(design, 'R_43', pin_opt_R_43)

# gui.rebuild()
# gui.autoscale()

In [49]:
# R-PF coupler 43
coupler_length_43 = 95e-3

T_coupler_R_43 = LineTee_sqnl(design, 'T_coupler_R_43', options=dict(pos_x='2.740mm',
                                             pos_y='1.39mm',
                                             t_length=coupler_length_43,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '50um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 180
                                             ))

gui.rebuild()
gui.autoscale()

## Control lines

In [50]:
design.overwrite_enabled = True

options_c = Dict(
    total_length='1.5mm',
    hfss_wire_bonds = True,
    lead=Dict(
        start_straight='300um',
        end_straight='300um'),
    fillet='90um',
    trace_width='30um',
    trace_gap='18um')

control_Q1 = RouteStraight(design, 'control_Q1', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='Q_1',
            pin='control'),
        end_pin=Dict(
            component='launch_TL_read_12',
            pin='tie')),
            **options_c))
control_Q2 = RouteStraight(design, 'control_Q2', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='Q_2',
            pin='control'),
        end_pin=Dict(
            component='launch_TL_read_22',
            pin='tie')),
            **options_c))
control_Q3 = RouteStraight(design, 'control_Q3', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='Q_3',
            pin='control'),
        end_pin=Dict(
            component='launch_TL_read_32',
            pin='tie')),
            **options_c))
control_Q4 = RouteStraight(design, 'control_Q4', options=Dict(
    pin_inputs=Dict(
        start_pin=Dict(
            component='Q_4',
            pin='control'),
        end_pin=Dict(
            component='launch_TL_read_42',
            pin='tie')),
            **options_c))
gui.rebuild()
gui.autoscale()

# Analyze

## Capacitance LOM

In [81]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [82]:
q3d = c1.sim.renderer

In [83]:
q3d.start()

INFO 04:56PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:56PM [load_ansys_project]: 	Opened Ansys App
INFO 04:56PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 04:56PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Luxcuse/Documents/Ansoft/
	Project:   Project9
INFO 04:56PM [connect_design]: 	Opened active design
	Design:    Q_1_q3d [Solution type: Q3D]
INFO 04:56PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:56PM [connect]: 	Connected to project "Project9" and design "Q_1_q3d" 😀 



True

In [84]:
q3d.activate_ansys_design("reset_q3d", 'capacitive')

04:56PM 18s WARNING [activate_ansys_design]: The design_name=reset_q3d was not in active project.  Designs in active project are: 
['Reset_chip_ver4_1_hfss', 'Q_1_q3d'].  A new design will be added to the project.  
INFO 04:56PM [connect_design]: 	Opened active design
	Design:    reset_q3d [Solution type: Q3D]
WARNING 04:56PM [connect_setup]: 	No design setup detected.
WARNING 04:56PM [connect_setup]: 	Creating Q3D default setup.
INFO 04:56PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)


In [85]:
q3d.render_design(['Q_1'], [('Q_1','readout1')])

In [86]:
q3d.analyze_setup("Setup")

INFO 04:56PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:56PM [analyze]: Analyzing setup Setup


In [87]:
c1.sim.capacitance_matrix, c1.sim.units = q3d.get_capacitance_matrix()
c1.sim.capacitance_all_passes, _ = q3d.get_capacitance_all_passes()
c1.sim.capacitance_matrix

INFO 04:57PM [get_matrix]: Exporting matrix data to (C:\Users\Luxcuse\AppData\Local\Temp\tmpj1c4yg5k.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:57PM [get_matrix]: Exporting matrix data to (C:\Users\Luxcuse\AppData\Local\Temp\tmprn0kd8gb.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:57PM [get_matrix]: Exporting matrix data to (C:\Users\Luxcuse\AppData\Local\Temp\tmptbyuz4if.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 04:57PM [get_matrix]: Exporting matrix data to (C:\Users\Luxcuse\AppData\Local\Temp\tmpl5uvub2w.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 3, False
INFO 04:57PM [get_matrix]: Exporting matrix data to (C:\Users\Luxcuse\AppData\Local\Temp\tmp0mhsrl42.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 4, False


,control_connector_arm_Q_1,ground_main_plane,pad_bot_Q_1,pad_top_Q_1,readout1_connector_arm_Q_1,readout2_connector_arm_Q_1
control_connector_arm_Q_1,26.09397,-23.35507,-0.31654,-2.01000,-0.08682,-0.04274
ground_main_plane,-23.35507,269.95586,-65.32676,-64.18375,-28.74025,-28.70469
pad_bot_Q_1,-0.31654,-65.32676,116.16049,-30.13103,-4.50949,-11.91042
pad_top_Q_1,-2.01000,-64.18375,-30.13103,116.59703,-11.87804,-4.51629
readout1_connector_arm_Q_1,-0.08682,-28.74025,-4.50949,-11.87804,46.05488,-0.06739
readout2_connector_arm_Q_1,-0.04274,-28.70469,-11.91042,-4.51629,-0.06739,46.03880


In [89]:
c1.setup.junctions=Dict(Lj=15.2, Cj=2)
c1.setup.freq_readout = 6.6
c1.setup.freq_bus = [5.5,6.3]

c1.run_lom()
c1.lumped_oscillator_all

[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.993757 [GHz]
EC 497.053023 [MHz]
EJ 10.749711 [GHz]
alpha -639.451529 [MHz]
dispersion 5806.365398 [KHz]
Lq 15.193871 [nH]
Cq 38.970144 [fF]
T1 20.232964 [us]

**Coupling Properties**

tCqbus1 1.073159 [fF]
gbus1_in_MHz 21.397571 [MHz]
χ_bus1 -0.779258 [MHz]
1/T1bus1 744.949356 [Hz]
T1bus1 213.645319 [us]

tCqbus2 0.434928 [fF]
gbus2_in_MHz 7.254220 [MHz]
χ_bus2 0.935000 [MHz]
1/T1bus2 12.913745 [Hz]
T1bus2 12324.460508 [us]

tCqbus3 -5.618136 [fF]
gbus3_in_MHz -105.495338 [MHz]
χ_bus3 -49.245180 [MHz]
1/T1bus3 7108.257842 [Hz]
T1bus3 22.390148 [us]
Bus-Bus Couplings
gbus1_2 0.246237 [MHz]
gbus1_3 73.383450 [MHz]
gbus2_3 48.318631 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.41946,579.660853,10.749711,-772.653472,14390.503261,"[20.860152031688333, 9.242476502575649, -132.8...","[-3.9116941633721396, -0.9789326078273229, 349...",3.911694,20.860152
2,6.246167,545.052188,10.749711,-715.7339,10099.671444,"[20.68830274368252, 6.795651684273803, -100.12...","[-1.6228521166525156, -2.911610894384402, -346...",1.622852,20.688303
3,6.136438,523.84022,10.749711,-681.64055,7985.509181,"[21.2704809433785, 7.0229972983211475, -101.07...","[-1.1658662968045936, 2.3367484586715626, -100...",1.165866,21.270481
4,6.071552,511.548195,10.749711,-662.161388,6921.425881,"[21.43788489542823, 7.088605549632647, -102.75...","[-0.9713558695021932, 1.284427097730223, -68.8...",0.971356,21.437885
5,6.052424,507.959678,10.749711,-656.512979,6631.689448,"[21.415619535141552, 7.223578327210761, -103.8...","[-0.9173051036233117, 1.1909716480513277, -63....",0.917305,21.415620
6,6.02844,503.482925,10.749711,-649.490674,6283.01068,"[21.306721333727314, 7.198276060449464, -104.5...","[-0.8488677155691196, 1.0516624714821556, -56....",0.848868,21.306721
7,6.011357,500.309442,10.749711,-644.528945,6044.252823,"[21.384469698817217, 7.264781061688502, -105.3...","[-0.8159772630507813, 0.9984927292006793, -53....",0.815977,21.384470
8,5.993757,497.053023,10.749711,-639.451529,5806.365398,"[21.397571001896047, 7.25422034860785, -105.49...","[-0.7792577925393925, 0.9350001650651922, -49....",0.779258,21.397571


## Eigenmode and EPR

In [51]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

In [52]:
eig_qb.sim.renderer.options['wb_size'] = 5

In [53]:
em_p = eig_qb.sim.setup

In [54]:
em_p.name = 'Reset_chip_ver4_5'
em_p.min_freq_ghz = 2
em_p.max_freq_ghz = 8
em_p.n_modes = 3
em_p.max_passes = 8
em_p.max_delta_f = 0.03
em_p.min_converged = 3
em_p.pct_refinement = 100
# Design variables can also be added in for direct simulation sweeps.
em_p.vars = Dict({'Lj1': '15.2 nH', 'Cj1': '0 fF'})

eig_qb.sim.setup

{'name': 'Reset_chip_ver4_5',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 2,
 'n_modes': 3,
 'max_delta_f': 0.03,
 'max_passes': 8,
 'min_passes': 1,
 'min_converged': 3,
 'pct_refinement': 100,
 'basis_order': 1,
 'vars': {'Lj1': '15.2 nH', 'Cj1': '0 fF'},
 'max_freq_ghz': 8}

In [55]:
eig_qb.sim.run(name="Reset_chip_ver4_1", 
               components=['Q_1', 'PF_11', 'R_11', 'T_coupler_PF_11', 'T_coupler_R_11', 'highC_PF_TL_11', 'TL_11', 'launch_TL_read_11'], 
               open_terminations=[])

INFO 03:54PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:54PM [load_ansys_project]: 	Opened Ansys App
INFO 03:54PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 03:54PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Luxcuse/Documents/Ansoft/
	Project:   Project9
INFO 03:54PM [connect_design]: 	Opened active design
	Design:    Reset_chip_ver4_1_hfss [Solution type: Eigenmode]
INFO 03:54PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:54PM [connect]: 	Connected to project "Project9" and design "Reset_chip_ver4_1_hfss" 😀 

INFO 03:54PM [connect_design]: 	Opened active design
	Design:    Reset_chip_ver4_1_hfss [Solution type: Eigenmode]
INFO 03:54PM [get_setup]: 	Opened setup `Reset_chip_ver4_5`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:54PM [analyze]: Analyzing setup Reset_chip_ver4_5
03:57PM 06s INFO [get_f_convergence]: Saved convergences to c:\Users\Luxcuse\Documents\GitHub\qiskit-metal-SQNL\hfss_eig_

In [56]:
eig_qb.sim.plot_convergences()

In [57]:
eig_qb.del_junction()
eig_qb.add_junction('jj1', 'Lj1', 'Cj1', rect='JJ_rect_Lj_Q_1_rect_jj', line='JJ_Lj_Q_1_rect_jj_')
# eig_qb.add_junction('jj2', 'Lj2', 'Cj2', rect='JJ_rect_Lj_Q_2_rect_jj', line='JJ_Lj_Q_2_rect_jj_')
eig_qb.setup.sweep_variable = 'Lj1'
eig_qb.setup

{'junctions': {'jj1': {'Lj_variable': 'Lj1',
   'Cj_variable': 'Cj1',
   'rect': 'JJ_rect_Lj_Q_1_rect_jj',
   'line': 'JJ_Lj_Q_1_rect_jj_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj1'}

In [58]:
eig_qb.run_epr()
# (pyEPR allows to switch modes: eprd.set_mode(1))

Design "Reset_chip_ver4_1_hfss" info:
	# eigenmodes    3
	# variations    1
Design "Reset_chip_ver4_1_hfss" info:
	# eigenmodes    3
	# variations    1

        energy_elec_all       = 9.63159497953955e-25
        energy_elec_substrate = 8.84600897547531e-25
        EPR of substrate = 91.8%

        energy_mag    = 9.62365546833583e-25
        energy_mag % of energy_elec_all  = 99.9%
        

Variation 0  [1/1]

  Mode 0 at 4.86 GHz   [1/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.2%  4.083e-24  3.43e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 97.25%
	jj1             0.912634  (+)        0.0258391
		(U_tot_cap-U_tot_ind)/mean=5.38%
Calculating Qdielectric_main for mode 0 (0/2)
p_dielectric_main_0 = 0.9209499841850147

  Mode 1 at 6.32 GHz   [2/3]
    Calculating ℰ_magnetic,ℰ_elect

WARNING 03:57PM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project9\Reset_chip_ver4_1_hfss\2024-03-20 15-57-17.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization


  result['Q_coupling'] = self.Qm_coupling[variation][self.Qm_coupling[variation].columns[junctions]][modes]#TODO change the columns to junctions

  result['Qs'] = self.Qs[variation][self.PM[variation].columns[junctions]][modes] #TODO change the columns to junctions



Pm_norm=
modes
0    1.114342
1    1.067321
2    1.069593
dtype: float64

Pm_norm idx =
     jj1
0   True
1  False
2  False
*** P (participation matrix, not normlz.)
        jj1
0  0.889646
1  0.000759
2  0.000645

*** S (sign-bit matrix)
   s_jj1
0      1
1      1
2     -1
*** P (participation matrix, normalized.)
      0.99
   0.00076
   0.00064

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       269    0.537    0.478
     0.537 0.000268 0.000476
     0.478 0.000476 0.000212

*** Chi matrix ND (MHz) 
       305     0.37    0.354
      0.37 0.000139 0.000262
     0.354 0.000262 0.000124

*** Frequencies O1 PT (MHz)
0    4587.060147
1    6323.987672
2    6621.457172
dtype: float64

*** Frequencies ND (MHz)
0    4569.851671
1    6324.016930
2    6621.477393
dtype: float64

*** Q_coupling
Empty DataFrame
Columns: []
Index: [0, 1, 2]


...

###### Numerical diagonalization

Lj1,15.2
0,4569.85
1,6324.02
2,6621.48


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1         2
Lj1                               
15.2 0  305.06  3.70e-01  3.54e-01
     1    0.37  1.39e-04  2.62e-04
     2    0.35  2.62e-04  1.24e-04

In [59]:
eig_qb.sim.close()

# 360

In [11]:
Phi_0 = 2.067833848e-15
Lj_test = 13.2
Ic_test = Phi_0/2/np.pi/Lj_test/1e-9
print("When Lj={:.3f}[nH],".format(Lj_test)+"Ic={:.3f}[nA]".format(Ic_test/1e-9))

When Lj=13.200[nH],Ic=24.932[nA]


In [13]:
# Calculate RRR
R_300K = 11730
Ic = 24.932e-9
Delta_Al = 2.1e-4

RRR = Ic * R_300K * 2 / np.pi / Delta_Al

print('RRR={}'.format(RRR))

RRR=1.0951939955150338


In [14]:
print('Rn={}'.format(np.pi/2*Delta_Al/Ic))

Rn=10710.431255134636
